<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/GPU_QM_DM_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
import tensorflow as tf
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [3]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


## Baseline

In [7]:
BATCH_SIZE = 256

model = tf.keras.models.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(10),
tf.keras.layers.Softmax()
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)
EPOCHS = 10
  
history = model.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 1s 3ms/step - loss: 0.8394 - categorical_accuracy: 0.7736 - val_loss: 0.2391 - val_categorical_accuracy: 0.9353
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.2306 - categorical_accuracy: 0.9348 - val_loss: 0.1806 - val_categorical_accuracy: 0.9508
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1758 - categorical_accuracy: 0.9490 - val_loss: 0.1545 - val_categorical_accuracy: 0.9574
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1393 - categorical_accuracy: 0.9612 - val_loss: 0.1345 - val_categorical_accuracy: 0.9613
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.1132 - categorical_accuracy: 0.9675 - val_loss: 0.1216 - val_categorical_accuracy: 0.9658
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0950 - categorical_accuracy: 0.9746 - val_loss: 0.1127 - val_categorical_accuracy: 0.9684
Epoc

## Full training

In [45]:
BATCH_SIZE = 256
input_dim = 784
component_dim = 512
gamma = 2**-5
n_comp = 80
random_state = 0
dim_h = 30

inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=dim_h, n_comp=n_comp)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMClassifSDecompFDMatrix(dim_x=dim_h, dim_y=10, n_comp=n_comp)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

# fm_x1.trainable = False

EPOCHS = 10
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 14s 72ms/step - loss: 1.3676 - accuracy: 0.6204 - val_loss: 0.4025 - val_accuracy: 0.9316
Epoch 2/10
188/188 [==============================] - 13s 70ms/step - loss: 0.3497 - accuracy: 0.9428 - val_loss: 0.3187 - val_accuracy: 0.9455
Epoch 3/10
188/188 [==============================] - 14s 72ms/step - loss: 0.2756 - accuracy: 0.9571 - val_loss: 0.3494 - val_accuracy: 0.9398
Epoch 4/10
188/188 [==============================] - 13s 71ms/step - loss: 0.2759 - accuracy: 0.9571 - val_loss: 0.3264 - val_accuracy: 0.9461
Epoch 5/10
188/188 [==============================] - 13s 70ms/step - loss: 0.2513 - accuracy: 0.9622 - val_loss: 0.2629 - val_accuracy: 0.9561
Epoch 6/10
188/188 [==============================] - 13s 70ms/step - loss: 0.2041 - accuracy: 0.9714 - val_loss: 0.2457 - val_accuracy: 0.9589
Epoch 7/10
188/188 [==============================] - 13s 70ms/step - loss: 0.1885 - accuracy: 0.9745 - val_loss: 0.2371 - val_accuracy:

## Layerwise training

In [46]:
BATCH_SIZE = 256
input_dim = 784
component_dim = 512
gamma = 2**-5
n_comp = 80
random_state = 0

inputs = tf.keras.Input(shape=(784,))
fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
#qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig, num_eig=num_eig)
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_y = qmdmc(rho_x)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
  
history = qmdmclf.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 12s 59ms/step - loss: 1.2762 - accuracy: 0.6820 - val_loss: 0.3678 - val_accuracy: 0.9387
Epoch 2/10
188/188 [==============================] - 10s 54ms/step - loss: 0.3325 - accuracy: 0.9466 - val_loss: 0.2920 - val_accuracy: 0.9494
Epoch 3/10
188/188 [==============================] - 10s 54ms/step - loss: 0.2429 - accuracy: 0.9633 - val_loss: 0.2647 - val_accuracy: 0.9548
Epoch 4/10
188/188 [==============================] - 10s 54ms/step - loss: 0.2071 - accuracy: 0.9717 - val_loss: 0.9818 - val_accuracy: 0.7632
Epoch 5/10
188/188 [==============================] - 10s 54ms/step - loss: 0.6166 - accuracy: 0.8691 - val_loss: 0.3194 - val_accuracy: 0.9459
Epoch 6/10
188/188 [==============================] - 10s 54ms/step - loss: 0.2694 - accuracy: 0.9602 - val_loss: 0.2970 - val_accuracy: 0.9493
Epoch 7/10
188/188 [==============================] - 10s 54ms/step - loss: 0.3212 - accuracy: 0.9506 - val_loss: 0.2907 - val_accuracy:

In [47]:
inputs = tf.keras.Input(shape=(784,))
fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
psi_x = fm_x1(inputs)
ones = tf.ones_like(inputs[:, 0:1])
rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
rho_x = tf.expand_dims(rho_x, axis=-1)
qmdmc1 = layers.QMClassifSDecompFDMatrix(dim_x=component_dim, dim_y=10, n_comp=n_comp)
rho_h = qmdmc1(rho_x)
qmdmc2 = layers.QMClassifSDecompFDMatrix(dim_x=10, dim_y=10, n_comp=n_comp)
rho_y = qmdmc2(rho_h)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

fm_x1.set_weights(fm_x.get_weights())
fm_x1.trainable = False
qmdmc1.set_weights(qmdmc.get_weights())
qmdmc1.trainable = False
EPOCHS = 5
  
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/5
188/188 [==============================] - 13s 66ms/step - loss: 1.0803 - accuracy: 0.8844 - val_loss: 0.2337 - val_accuracy: 0.9626
Epoch 2/5
188/188 [==============================] - 12s 66ms/step - loss: 0.1579 - accuracy: 0.9830 - val_loss: 0.2100 - val_accuracy: 0.9642
Epoch 3/5
188/188 [==============================] - 12s 64ms/step - loss: 0.1356 - accuracy: 0.9867 - val_loss: 0.2037 - val_accuracy: 0.9656
Epoch 4/5
188/188 [==============================] - 12s 64ms/step - loss: 0.1219 - accuracy: 0.9890 - val_loss: 0.1886 - val_accuracy: 0.9679
Epoch 5/5
188/188 [==============================] - 12s 65ms/step - loss: 0.1105 - accuracy: 0.9910 - val_loss: 0.1837 - val_accuracy: 0.9690


In [48]:
fm_x1.trainable = True
qmdmc1.trainable = True
opt.learning_rate = 0.0005
EPOCHS = 10
history = qmdmclf2.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 12s 65ms/step - loss: 0.1018 - accuracy: 0.9922 - val_loss: 0.1816 - val_accuracy: 0.9688
Epoch 2/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0982 - accuracy: 0.9931 - val_loss: 0.1789 - val_accuracy: 0.9691
Epoch 3/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0950 - accuracy: 0.9933 - val_loss: 0.1780 - val_accuracy: 0.9696
Epoch 4/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0919 - accuracy: 0.9941 - val_loss: 0.1762 - val_accuracy: 0.9703
Epoch 5/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0890 - accuracy: 0.9943 - val_loss: 0.1740 - val_accuracy: 0.9709
Epoch 6/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0860 - accuracy: 0.9946 - val_loss: 0.1734 - val_accuracy: 0.9707
Epoch 7/10
188/188 [==============================] - 12s 65ms/step - loss: 0.0834 - accuracy: 0.9951 - val_loss: 0.1718 - val_accuracy: